In [1]:
import numpy as np
import h5py
import pylab as pl
import arrayfire as af
from scipy.integrate import odeint
import scipy.fftpack as ff
from scipy import interpolate

In [2]:
pl.rcParams['figure.figsize']  = 12, 7.5
pl.rcParams['lines.linewidth'] = 1.5
pl.rcParams['font.family']     = 'serif'
pl.rcParams['font.weight']     = 'bold'
pl.rcParams['font.size']       = 20
pl.rcParams['font.sans-serif'] = 'serif'
pl.rcParams['text.usetex']     = True
pl.rcParams['axes.linewidth']  = 1.5
pl.rcParams['axes.titlesize']  = 'medium'
pl.rcParams['axes.labelsize']  = 'medium'

pl.rcParams['xtick.major.size'] = 8
pl.rcParams['xtick.minor.size'] = 4
pl.rcParams['xtick.major.pad']  = 8
pl.rcParams['xtick.minor.pad']  = 8
pl.rcParams['xtick.color']      = 'k'
pl.rcParams['xtick.labelsize']  = 'medium'
pl.rcParams['xtick.direction']  = 'in'

pl.rcParams['ytick.major.size'] = 8
pl.rcParams['ytick.minor.size'] = 4
pl.rcParams['ytick.major.pad']  = 8
pl.rcParams['ytick.minor.pad']  = 8
pl.rcParams['ytick.color']      = 'k'
pl.rcParams['ytick.labelsize']  = 'medium'
pl.rcParams['ytick.direction']  = 'in'

### Linear Theory code begins here

$$\mathrm{Setting\;the\;variables\;for\;the\;maxwell\;distribution}$$
$$\sqrt{\left(m/2\pi KT\right)}e^{\left(-mv^{2}/2KT\right)}$$

In [3]:
# Setting the variables in the maxwell distribution
m = 1
K = 1
T = 1
e = -10

In [4]:
# k for the mode in fourier space
k = 2*np.pi
amp = 0.5

$f_0$ is given by
$$\sqrt{\left(m/2\pi KT\right)}e^{\left(-mv^{2}/2KT\right)}$$
$\frac{\partial f_{0}}{\partial v}$ is given by
$$\sqrt{\left(m/2\pi KT\right)}e^{\left(-mv^{2}/2KT\right)}(-mv/KT)$$

In [5]:
me = 1
# The maxwell Boltzman function
def f_0(v):
    return np.sqrt(me/(2*np.pi*K*T))*np.exp(-me*v**2/(2*K*T))

# This the function which returns the derivative of the maxwell boltzmann equation
def diff_f_0_v(v):
    return np.sqrt(me/(2*np.pi*K*T))*np.exp(-me*v**2/(2*K*T)) * ( -me * v / (K * T))

In [6]:
# Assign the maxim and minimum velocity for the velocity grid
velocity_max =  +10
velocity_min =  -10

In [7]:
# Set the divisions for the velocity grid
number_of_velocities_points = 501
velocity_x                  = np.linspace(velocity_min, velocity_max, number_of_velocities_points)
dv                          = velocity_x[1] - velocity_x[0]

$$\mathrm{Equations\;used\;for\;the\;first\;approach}$$
\begin{align}
\frac{\partial \left(\delta \hat{f}\left(v, t\right)\right)}{\partial t}+ik\;v_{x} \cdot \delta \hat{f}\left(v, t\right)+\left(\frac{q^2}{imk}\left(\int\delta \hat{f}\left(v, t\right).dv\right)\right)\left(\frac{\partial\left(f_{0}\right)}{\partial v}\right)=0
\end{align}

The above equation can be split into a coupled set of ordinary differential equations in the manner shown below: \\
Let the real and imaginary parts of $\delta \hat{f}\left(v, t\right)$ be denoted by $f_{r}$ and $f_{i}$ respectively. This results in the following set of equations: \\
\begin{align}
\frac{\partial f_{r}}{\partial t}=kv_{x}f_{i}-\frac{q^{2}}{mk}\left(\int f_{i}dv\right)\frac{\partial f_{0}}{\partial v_{x}} \\
\frac{\partial f_{i}}{\partial t}=-kv_{x}f_{r}+\frac{q^{2}}{mk}\left(\int f_{r}dv_{x}\right)\frac{\partial f_{0}}{\partial v_{x}}
\end{align}

In [8]:
# Function that returns df_i/dt and df_r/dt used for odeint function
# See the latex document for more details on the differential equations
# This has been done to split the imaginary and real part of the ODE
def diff_delta_f(Y,t):
    f_r = Y[0:len(velocity_x)]  # Initial conditions for odeint
    f_i = Y[len(velocity_x): 2 * len(velocity_x)]

    int_Df_i = np.sum(f_i) * (velocity_x[1]-velocity_x[0])
    int_Df_r = np.sum(f_r) * (velocity_x[1]-velocity_x[0])

    # This the derivate for f_r and f_i given in the latex document
    dYdt =np.concatenate([(k * velocity_x * f_i) - e*e*(1/m)*(int_Df_i * diff_f_0_v(velocity_x)/(m * k) ), \
                           -(k * velocity_x * f_r) + e*e*(1/m)*(int_Df_r * diff_f_0_v(velocity_x)/(m * k) )\
                         ], axis = 0)
    # This returns the derivative for the coupled set of ODE

    return dYdt

$$\mathrm{Equations\;used\;for\;the\;second\;approach}$$
\begin{align}
\frac{\partial\left(\delta\hat{f}\left(v,t\right)\right)}{\partial t}+ikv_{x}\delta\hat{f}\left(v,t\right)+\left(\int\int \frac{q^{2}}{m} v_{x}\delta f(v,t)dv_{x}dt\right)\left(\frac{\partial\left(f_{0}\right)}{\partial v_{x}}\right)=0
\end{align}
Let the real and imaginary of $\delta \hat{f}(v,t)$ be $f_r$ and $f_i$ respectively. The set of coupled differential equations to split the above equation into its real and imaginary components that can be derived from these are:
\begin{align}
\frac{\partial f_{r}}{\partial t}=kv_{x}f_{i}-\frac{q}{m}E_{r}\frac{\partial f_{0}}{\partial x} \\
\frac{\partial E_{r}}{\partial t}=-\int q v_{x}f_{r}dv_{x} \\
\frac{\partial f_{i}}{\partial t}=-kv_{x}f_{r}-\frac{q}{m}E_{i}\frac{\partial f_{0}}{\partial x} \\
\frac{\partial E_{i}}{\partial t}=-\int q v_{x}f_{i}dv_{x}
\end{align}

In [9]:
def diff_delta_f_Ex(Y,t):

    f_r = Y[0:len(velocity_x)]  # Initial conditions for odeint
    f_i = Y[len(velocity_x): 2 * len(velocity_x)]
    E_x_r = Y[2 * len(velocity_x)]
    E_x_i = Y[2 * len(velocity_x) + 1]

    int_v_delta_f_dv_i = e * np.sum(f_i * velocity_x) * (dv)
    int_v_delta_f_dv_r = e * np.sum(f_r * velocity_x) * (dv)
    int_v_delta_f_dv = np.array([int_v_delta_f_dv_r, int_v_delta_f_dv_i ] )

    # This the derivate for f_r and f_i given in the latex document
    dYdt =np.concatenate([(    k * velocity_x * f_i) - e*(1/m)*(E_x_r * diff_f_0_v(velocity_x) ), \
                            - (k * velocity_x * f_r) - e*(1/m)*(E_x_i * diff_f_0_v(velocity_x) ), \
                                -1 * int_v_delta_f_dv\
                         ], axis = 0\
                        )
    # This returns the derivative for the coupled set of ODE

    return dYdt

In [10]:
# Set the initial conditions for delta f(v,t) here
delta_f_initial = np.zeros((2 * len(velocity_x)), dtype = np.float)
delta_f_initial[0: len(velocity_x)] = amp * f_0(velocity_x)

delta_f_Ex_initial = np.zeros((2 * len(velocity_x)+2), dtype = np.float)
delta_f_Ex_initial[0 : len(velocity_x)] = amp * (e * e / m)*f_0(velocity_x)
delta_f_Ex_initial[2 * len(velocity_x) + 1] = -1 * (e * e / m) * (1/k) * np.sum(delta_f_Ex_initial[0: len(velocity_x)] ) * dv

In [11]:
# Setting the parameters for time here
final_time = 3
dt = 0.001
time_ana = np.arange(0, final_time, dt)

In [12]:
# Variable for temperorily storing the real and imaginary parts of delta f used for odeint
initial_conditions_delta_f = np.zeros((2 * len(velocity_x)), dtype = np.float)
old_delta_f = np.zeros((2 * len(velocity_x)), dtype = np.float)


initial_conditions_delta_f_Ex = np.zeros((2 * len(velocity_x) + 2), dtype = np.float)
old_delta_f_Ex = np.zeros((2 * len(velocity_x) + 2 ), dtype = np.float)
# Variable for storing delta rho

In [13]:
delta_rho_kspace_method1  = np.zeros(len(time_ana), dtype = np.float)
delta_rho_kspace_method2  = np.zeros(len(time_ana), dtype = np.float)
Ex_real_amplitude         = np.zeros(len(time_ana), dtype = np.float)
Ex_imag_amplitude         = np.zeros(len(time_ana), dtype = np.float)
Ex_amplitude              = np.zeros(len(time_ana), dtype = np.float)
delta_f_temp              = np.zeros(2 * len(velocity_x), dtype=np.float)
temperory_delta_f_Ex      = np.zeros(2 * len(velocity_x) + 2, dtype=np.float)

In [14]:
for time_index, t0 in enumerate(time_ana):
    if(time_index%1000==0):
        print("Computing for time = ", time_index)
        
    t0 = time_ana[time_index]
    
    if (time_index == time_ana.size - 1):
        break
    t1 = time_ana[time_index + 1]
    t = [t0, t1]

    # delta f is defined on the velocity grid


    # Initial conditions for the odeint
    if(time_index == 0):
        # Initial conditions for the odeint for the 2 ODE's respectively for the first time step
        # First column for storing the real values of delta f and 2nd column for the imaginary values
        initial_conditions_delta_f                 = delta_f_initial.copy()
        initial_conditions_delta_f_Ex                 = delta_f_Ex_initial.copy()
        # Storing the integral sum of delta f dv used in odeint

    else:
        # Initial conditions for the odeint for the 2 ODE's respectively for all other time steps
        # First column for storing the real values of delta f and 2nd column for the imaginary values
        initial_conditions_delta_f= old_delta_f.copy()
        initial_conditions_delta_f_Ex= old_delta_f_Ex.copy()
        # Storing the integral sum of delta f dv used in odeint

    # Integrating delta f

    temperory_delta_f = odeint(diff_delta_f, initial_conditions_delta_f, t)[1]
    temperory_delta_f_Ex = odeint(diff_delta_f_Ex, initial_conditions_delta_f_Ex, t)[1]

    # Saving delta rho for current time_index
    delta_rho_kspace_method1[time_index] = ((sum(dv * temperory_delta_f[ 0: len(velocity_x)])))
    delta_rho_kspace_method2[time_index] = ((sum(dv * temperory_delta_f_Ex[ 0: len(velocity_x)])))
    Ex_real_amplitude[time_index] = (e/k)*sum(  dv * temperory_delta_f[ 0: len(velocity_x)]  )
    Ex_imag_amplitude[time_index] = (e/k)*(sum  (  dv * temperory_delta_f_Ex[ 1 * len(velocity_x) : 2 * len(velocity_x)]  ))
    Ex_amplitude[time_index] = np.sqrt( Ex_real_amplitude[time_index]**2 + Ex_imag_amplitude[time_index]**2  )
    # Saving the solution for to use it for the next time step
    old_delta_f = temperory_delta_f.copy()
    old_delta_f_Ex = temperory_delta_f_Ex.copy()

Computing for time =  0
Computing for time =  1000
Computing for time =  2000


In [15]:
h5f = h5py.File('LT.h5', 'w')
h5f.create_dataset('time_LT', data = time_ana)
h5f.create_dataset('Ex_amp', data = (abs(Ex_amplitude)))
h5f.close()